In [ ]:
# Vergleich: Datenverarbeitung mit Spark vs. Pandas
# -------------------------------------------------
#
# Ziel dieses Notebooks:
# - Vergleich der Datenverarbeitungsgeschwindigkeit zwischen Apache Spark und Pandas.
# - Analyse, bei welchen Datenmengen Pandas an seine Grenzen stößt und Spark seine Vorteile zeigt.
#
# Überblick:
# - Der erste Codeblock implementiert die Datenverarbeitung mit Spark.
# - Der zweite Codeblock implementiert die gleiche Logik mit Pandas.
# - Beide Ansätze werden in Bezug auf Laufzeit und Speicherbedarf verglichen.
#
# Voraussetzungen:
# - Python-Version: 3.8 oder höher
# - Apache Spark: 3.5.0
# - Installierte Bibliotheken: pyspark, pandas, requests, folium
#
# Ablauf:
# 1. Daten herunterladen und vorbereiten.
# 2. Daten mit beiden Ansätzen verarbeiten.
# 3. Laufzeit messen und vergleichen.
#
# Starten wir mit dem Import der notwendigen Bibliotheken und dem Setup der Spark-Umgebung.

In [12]:
from pyspark.sql import SparkSession
import requests
from io import BytesIO
import zipfile
from concurrent.futures import ThreadPoolExecutor
import tempfile
import os
from pyspark.sql.functions import col, to_timestamp, count, lit
import folium  # Import für Kartenvisualisierung
import time  # Import für Zeitmessung

In [13]:
# Globale Konfigurationsvariablen
# Pfad für temporäre Speicherung
temp_storage_path = "./data/temp"  # Konfigurierbarer Speicherpfad für temporäre Dateien
os.makedirs(temp_storage_path, exist_ok=True)  # Ordner erstellen, falls er nicht existiert

In [14]:
# Liste der URLs
csv_urls = [
    "https://web.ais.dk/aisdata/aisdk-2024-03-01.zip",
    "https://web.ais.dk/aisdata/aisdk-2024-03-02.zip"]
  #  "https://web.ais.dk/aisdata/aisdk-2024-03-03.zip",
  #  "https://web.ais.dk/aisdata/aisdk-2024-03-04.zip",
  # "https://web.ais.dk/aisdata/aisdk-2024-03-05.zip"
#]

In [15]:
# Schritt 1: Spark-Session erstellen
spark = SparkSession.builder \
    .appName("AIS Data Processing") \
    .getOrCreate()

# Schritt 2: Funktion zum Herunterladen, Entpacken und Speichern von CSV-Dateien
def download_and_unzip_to_temp_csv(url):
    response = requests.get(url)
    response.raise_for_status()
    zipfile_bytes = BytesIO(response.content)
    with zipfile.ZipFile(zipfile_bytes, 'r') as z:
        csv_filename = z.namelist()[0]  # Der Name der CSV-Datei im ZIP-Archiv
        with z.open(csv_filename) as csv_file:
            temp_file_path = os.path.join(temp_storage_path, csv_filename)
            with open(temp_file_path, "wb") as temp_file:
                temp_file.write(csv_file.read())
            return temp_file_path

# Schritt 4: Paralleles Herunterladen und Speichern der CSV-Dateien in temporären Pfaden
start_time = time.time()  # Startzeitpunkt
with ThreadPoolExecutor(max_workers=10) as executor:
    csv_file_paths = list(executor.map(download_and_unzip_to_temp_csv, csv_urls))
end_time = time.time()  # Endzeitpunkt

# Zeitmessung für den Download
print(f"Die Zeit für das Herunterladen und Entpacken der Dateien beträgt: {end_time - start_time:.2f} Sekunden")


# Schritt 5: CSV-Dateien mit Spark einlesen und kombinieren
# Erstelle eine Liste von DataFrames für jede CSV-Datei
dataframes = [spark.read.csv(path, header=True, inferSchema=True) for path in csv_file_paths]

# Kombiniere alle DataFrames zu einem großen DataFrame
combined_df = dataframes[0]
for df in dataframes[1:]:
    combined_df = combined_df.union(df)

Die Zeit für das Herunterladen und Entpacken der Dateien beträgt: 406.66 Sekunden


In [16]:
# Schritt 6: Einige Beispielzeilen ausgeben, um mögliche MMSI-Nummern anzuzeigen
combined_df.show(10)

# Originale Anzahl der Einträge ausgeben
print(f"Der Originale Datensatz hat {combined_df.count()} Einträge.")


+-------------------+--------------+---------+---------+---------+--------------------+----+----+-----+-------+-------+--------+----+---------+----------+-----+------+------------------------------+-------+-----------+----+----------------+----+----+----+----+
|        # Timestamp|Type of mobile|     MMSI| Latitude|Longitude| Navigational status| ROT| SOG|  COG|Heading|    IMO|Callsign|Name|Ship type|Cargo type|Width|Length|Type of position fixing device|Draught|Destination| ETA|Data source type|   A|   B|   C|   D|
+-------------------+--------------+---------+---------+---------+--------------------+----+----+-----+-------+-------+--------+----+---------+----------+-----+------+------------------------------+-------+-----------+----+----------------+----+----+----+----+
|01/03/2024 00:00:00|       Class A|219000873| 56.99091|10.304543|Under way using e...|NULL| 0.0| 30.2|   NULL|Unknown| Unknown|NULL|Undefined|      NULL| NULL|  NULL|                     Undefined|   NULL|    Unknown

Der Originale Datensatz hat 31817670 Einträge.


In [17]:
########################################################
# 2. Basisstationen herausfiltern, da diese keine Navigationsdaten anzeigen ("Type of mobile" != "Base Station")
########################################################

# Prüfen, ob die Spalte "Type of mobile" existiert und filtern
if "Type of mobile" in combined_df.columns:
    combined_df = combined_df.filter(col("Type of mobile") != "Base Station")
else:
    print("Warnung: 'Type of mobile' Spalte nicht vorhanden, Überspringe diesen Schritt.")

print(f"Der angepasste Datensatz hat {combined_df.count()} Einträge.")

Der angepasste Datensatz hat 29508390 Einträge.


In [18]:
########################################################
# 3. Nur relevante Spalten behalten, um Datenmenge zu reduzieren
########################################################

relevant_columns = ["MMSI", "Latitude", "Longitude", "# Timestamp"]
combined_df = combined_df.select(*relevant_columns)

########################################################
# 4. Timestamp in datetime konvertieren
########################################################

combined_df = combined_df.withColumn("# Timestamp", to_timestamp(col("# Timestamp"), "dd/MM/yyyy HH:mm:ss"))

########################################################
# 5. Filtern von MMSI-Nummern, die genug Datenpunkte haben, damit vernünftige Routen angezeigt werden
########################################################

# Anzahl Datenpunkte pro MMSI bestimmen
mmsi_counts = combined_df.groupBy("MMSI").agg(count("*").alias("count"))

# Schwelle definieren (z.B. mindestens 50 Punkte)
threshold = 50
valid_mmsi = mmsi_counts.filter(col("count") >= threshold).select("MMSI").rdd.flatMap(lambda x: x).collect()

# Gefilterter DataFrame nur mit MMSI, die genügend Datenpunkte haben
filtered_by_count_df = combined_df.filter(col("MMSI").isin(valid_mmsi))

In [19]:
########################################################
# 6. Nach bestimmter MMSI und Zeitspanne filtern + Route plotten
########################################################

mmsi_number = 219016832  # Ersetze mit deiner MMSI

# Definiere Start- und Endzeitpunkt (im Format "dd/MM/yyyy HH:MM:SS")
start_str = "01/03/2024 00:00:00"  # Startzeitpunkt
end_str = "01/03/2024 06:59:59"    # Endzeitpunkt

# Konvertiere Start- und Endzeit in datetime-Objekte
start_dt = to_timestamp(lit(start_str), "dd/MM/yyyy HH:mm:ss")
end_dt = to_timestamp(lit(end_str), "dd/MM/yyyy HH:mm:ss")

# Prüfen, ob MMSI genug Daten hat
if mmsi_number not in valid_mmsi:
    print(f"MMSI {mmsi_number} hat nicht genügend Datenpunkte, um eine aussagekräftige Route anzuzeigen.")
else:
    # Nach MMSI und Zeitspanne filtern
    route_df = filtered_by_count_df.filter(
        (col("MMSI") == mmsi_number) &
        (col("# Timestamp") >= start_dt) &
        (col("# Timestamp") <= end_dt)
    ).orderBy("# Timestamp")

    # Überprüfen, ob gefilterte Daten vorhanden sind
    if route_df.count() == 0:
        print(f"Keine Daten für MMSI {mmsi_number} zwischen {start_str} und {end_str}")
    else:
        # Daten in Pandas konvertieren, um Karte zu erstellen
        pandas_df = route_df.toPandas()

        # Karte erstellen und Route plotten
        mean_lat = pandas_df["Latitude"].mean()
        mean_lon = pandas_df["Longitude"].mean()
        
        route_map = folium.Map(location=[mean_lat, mean_lon], zoom_start=8)
        
        # Koordinatenliste für PolyLine
        coords = pandas_df[["Latitude", "Longitude"]].values.tolist()
        
        # PolyLine hinzufügen
        folium.PolyLine(coords, color="blue", weight=2.5, opacity=1).add_to(route_map)
        
        # Karte speichern
        route_map.save("ship_route.html")
        print("Route wurde erfolgreich als 'ship_route.html' gespeichert.")

Route wurde erfolgreich als 'ship_route.html' gespeichert.
